`TTbarResCoffea` Notebook to perform the data-driven mistag-rate-based ttbar hadronic analysis. 
This module must be run twice: 
   1. Make the mistag rate in the "anti-tag and probe" selection 
and the expectation in the signal region from MC,
   1. Applies that mistag rate and the mod-mass procedure to the single-tag selection. 

These are all done in bins of
b-tag categories (0, 1, $\ge 2$) and rapidity ($|y| \le 1.0$, $|y| > 1.0$).
The signal region is two top-tagged jets. 
The background estimate is the single-tag selection weighted by the mistag rate from the
"anti-tag and probe" region, with the mass of the weighted jet set to a random
value from QCD MC in the 1-ttag region. 


The preselection is:
- AK4-based $H_{T} > 1100$ GeV (to be on the trigger plateau). 
- $\ge 2$ AK8 jets with AK8 $p_{T} > 400$ GeV and $|y| < 2.5$, loose jet ID applied from matched AK4 jets

The 1-tag selection adds:
- $\ge 1$ AK8 jet with top tagging applied to randomly-assigned tag jet. 


The anti-tag selection is disjoint from the 1-tag selection:
- $\ge 1$ AK8 jet with top tagging VETO applied to randomly-assigned tag jet. 


The 2-tag selection is:
- $\ge 2$ AK8 jets with top tagging applied to both leading jets. 


The ttbar candidate mass assumes the two leading top-tagged jets are the top quarks. 


In [ ]:
# ---- All Datasets used for analysis and dasgoclient searches ---- #

"""
/JetHT/Run2016B_ver1-Nano25Oct2019_ver1-v1/NANOAOD
/JetHT/Run2016B_ver2-Nano25Oct2019_ver2-v1/NANOAOD
/JetHT/Run2016C-Nano25Oct2019-v1/NANOAOD
/JetHT/Run2016D-Nano25Oct2019-v1/NANOAOD
/JetHT/Run2016E-Nano25Oct2019-v1/NANOAOD
/JetHT/Run2016F-Nano25Oct2019-v1/NANOAOD
/JetHT/Run2016G-Nano25Oct2019-v1/NANOAOD
/JetHT/Run2016H-Nano25Oct2019-v1/NANOAOD
/JetHT/Run2017B-Nano25Oct2019-v1/NANOAOD
/JetHT/Run2017C-Nano25Oct2019-v1/NANOAOD
/JetHT/Run2017D-Nano25Oct2019-v1/NANOAOD
/JetHT/Run2017E-Nano25Oct2019-v1/NANOAOD
/JetHT/Run2017F-Nano25Oct2019-v1/NANOAOD
/JetHT/Run2018A-Nano25Oct2019-v1/NANOAOD
/JetHT/Run2018B-Nano25Oct2019-v1/NANOAOD
/JetHT/Run2018C-Nano25Oct2019-v2/NANOAOD
/JetHT/Run2018D-Nano25Oct2019_ver2-v1/NANOAOD

/TTJets_TuneCP5_13TeV-amcatnloFXFX-pythia8/RunIIAutumn18NanoAODv7-Nano02Apr2020_102X_upgrade2018_realistic_v21_ext1-v1/NANOAODSIM
/QCD_Pt-15to7000_TuneCP5_Flat_13TeV_pythia8/RunIIFall17NanoAODv7-PU2017_12Apr2018_Nano02Apr2020_102X_mc2017_realistic_v8-v1/NANOAODSIM
"""

In [ ]:
import time
import copy
import scipy.stats as ss
from coffea import hist
from coffea.analysis_objects import JaggedCandidateArray
import coffea.processor as processor
from coffea import util
from awkward import JaggedArray
import numpy as np
import glob as glob
import itertools
import pandas as pd
from numpy.random import RandomState

In [ ]:
xrootdstr1 = 'root://cmseos.fnal.gov//'
xrootdstr2 = 'root://cmsxrootd.fnal.gov//'
xrootdstr3 = 'root://cmsxrootd-site.fnal.gov/'

In [ ]:
qcdfilename = 'QCD.txt'
with open(qcdfilename) as f:
    qcdfiles = [xrootdstr2 + s.strip() for s in f.readlines()]

In [ ]:
ttbarfilename = 'TTJets_TuneCP5_13TeV-amcatnloFXFX-pythia8.txt'
with open(ttbarfilename) as f:
    ttbarfiles = [xrootdstr2 + s.strip() for s in f.readlines()]

In [ ]:
zprimeDMfilename = 'ZprimeDMToTTbar.txt'
with open(zprimeDMfilename) as f:
    zprimeDMfiles = [xrootdstr2 + s.strip() for s in f.readlines()]

In [ ]:
jetdatafilename = 'JetHT_Data.txt'
with open(jetdatafilename) as f:
    jetdatafiles = [xrootdstr2 + s.strip() for s in f.readlines()[::3]] # Every third datafile
with open(jetdatafilename) as g:
    jetdatafiles2016 = [xrootdstr2 + s.strip() for s in g.readlines() if "/store/data/Run2016" in s]
with open(jetdatafilename) as h:
    jetdatafiles2017 = [xrootdstr2 + s.strip() for s in h.readlines() if "/store/data/Run2017" in s]
with open(jetdatafilename) as i:
    jetdatafiles2018 = [xrootdstr2 + s.strip() for s in i.readlines() if "/store/data/Run2018" in s]

In [ ]:
#print(jetdatafiles[2]) # Test to see if correct files are collected

In [ ]:
from columnservice.client import ColumnClient
cc = ColumnClient("coffea-dask.fnal.gov")
client = cc.get_dask()

In [ ]:
btag0_bins = []
ix = 400
while ix <= 7000:
    btag0_bins.append(ix)
    if ix < 1000:
        ix += 100
    elif ix < 1200:
        ix += 200
    elif ix < 1500:
        ix += 300
    elif ix < 2000:
        ix += 500
    elif ix < 3000:
        ix += 1000
    else:
        ix += 4000
print(btag0_bins)

In [ ]:
manual_bins = [400, 500, 600, 800, 1000, 1500, 2000, 3000, 7000]

In [ ]:

"""@TTbarResAnaHadronic Package to perform the data-driven mistag-rate-based ttbar hadronic analysis. 
"""
class TTbarResProcessor(processor.ProcessorABC):
    def __init__(self, prng, htCut=950., minMSD=105., maxMSD=210., tau32Cut=0.65, ak8PtMin=400., bdisc=0.8484,
                writePredDist=True,isData=True,year=2019, UseLookUpTables=False, lu=None, 
                ModMass=False, RandomDebugMode=False):
        
        self.prng = prng
        self.htCut = htCut
        self.minMSD = minMSD
        self.maxMSD = maxMSD
        self.tau32Cut = tau32Cut
        self.ak8PtMin = ak8PtMin
        self.bdisc = bdisc
        self.writePredDist = writePredDist
        self.writeHistFile = True
        self.isData = isData
        self.year=year
        self.UseLookUpTables = UseLookUpTables
        self.ModMass = ModMass
        self.RandomDebugMode = RandomDebugMode
        self.lu = lu # Look Up Tables
        
        self.ttagcats = ["Pt", "at", "pret", "0t", "1t", "1t+2t", "2t", "0t+1t+2t"] #anti-tag+probe, anti-tag, pre-tag, 0, 1, >=1, 2 ttags, any t-tag
        self.btagcats = ["0b", "1b", "2b"]   # 0, 1, >=2 btags
        self.ycats = ['cen', 'fwd']          # Central and forward
        # Combine categories like "0bcen", "0bfwd", etc:
        self.anacats = [ t+b+y for t,b,y in itertools.product( self.ttagcats, self.btagcats, self.ycats) ]
        print(self.anacats)
        
        dataset_axis = hist.Cat("dataset", "Primary dataset")
        cats_axis = hist.Cat("anacat", "Analysis Category")
        
        jetmass_axis = hist.Bin("jetmass", r"Jet $m$ [GeV]", 50, 0, 500)
        jetpt_axis = hist.Bin("jetpt", r"Jet $p_{T}$ [GeV]", 50, 0, 5000)
        ttbarmass_axis = hist.Bin("ttbarmass", r"$m_{t\bar{t}}$ [GeV]", 50, 0, 5000)
        jeteta_axis = hist.Bin("jeteta", r"Jet $\eta$", 50, -5, 5)
        jetphi_axis = hist.Bin("jetphi", r"Jet $\phi$", 50, -np.pi, np.pi)
        jety_axis = hist.Bin("jety", r"Jet $y$", 50, -3, 3)
        jetdy_axis = hist.Bin("jetdy", r"Jet $\Delta y$", 50, 0, 5)
        #jetp_axis = hist.Bin("jetp", r"Jet $p$ [GeV]", 100, 0, 10000)
        manual_axis = hist.Bin("jetp", r"Jet Momentum [GeV]", manual_bins)
        tagger_axis = hist.Bin("tagger", r"deepTag", 50, 0, 1)
        tau32_axis = hist.Bin("tau32", r"$\tau_3/\tau_2$", 50, 0, 2)
        
        subjetmass_axis = hist.Bin("subjetmass", r"SubJet $m$ [GeV]", 50, 0, 500)
        subjetpt_axis = hist.Bin("subjetpt", r"SubJet $p_{T}$ [GeV]", 50, 0, 2000)
        subjeteta_axis = hist.Bin("subjeteta", r"SubJet $\eta$", 50, -4, 4)
        subjetphi_axis = hist.Bin("subjetphi", r"SubJet $\phi$", 50, -np.pi, np.pi)

        self._accumulator = processor.dict_accumulator({
            'ttbarmass': hist.Hist("Counts", dataset_axis, cats_axis, ttbarmass_axis),
            
            'jetmass':         hist.Hist("Counts", dataset_axis, cats_axis, jetmass_axis),
            'SDmass':          hist.Hist("Counts", dataset_axis, cats_axis, jetmass_axis),
            'SDmass_precat':   hist.Hist("Counts", dataset_axis, jetpt_axis, jetmass_axis),
            
            'jetpt':     hist.Hist("Counts", dataset_axis, cats_axis, jetpt_axis),
            'jeteta':    hist.Hist("Counts", dataset_axis, cats_axis, jeteta_axis),
            'jetphi':    hist.Hist("Counts", dataset_axis, cats_axis, jetphi_axis),
            
            'probept':   hist.Hist("Counts", dataset_axis, cats_axis, jetpt_axis),
            'probep':    hist.Hist("Counts", dataset_axis, cats_axis, manual_axis),
            
            'jety':      hist.Hist("Counts", dataset_axis, cats_axis, jety_axis),
            'jetdy':     hist.Hist("Counts", dataset_axis, cats_axis, jetdy_axis),
            
            'deepTag_TvsQCD':   hist.Hist("Counts", dataset_axis, cats_axis, jetpt_axis, tagger_axis),
            'deepTagMD_TvsQCD': hist.Hist("Counts", dataset_axis, cats_axis, jetpt_axis, tagger_axis),
            
            'tau32':          hist.Hist("Counts", dataset_axis, cats_axis, tau32_axis),
            'tau32_2D':       hist.Hist("Counts", dataset_axis, cats_axis, jetpt_axis, tau32_axis),
            'tau32_precat': hist.Hist("Counts", dataset_axis, jetpt_axis, tau32_axis),
            
            'subjetmass':   hist.Hist("Counts", dataset_axis, cats_axis, subjetmass_axis),
            'subjetpt':     hist.Hist("Counts", dataset_axis, cats_axis, subjetpt_axis),
            'subjeteta':    hist.Hist("Counts", dataset_axis, cats_axis, subjeteta_axis),
            'subjetphi':    hist.Hist("Counts", dataset_axis, cats_axis, subjetphi_axis),
            
            'numerator':   hist.Hist("Counts", dataset_axis, cats_axis, manual_axis),
            'denominator': hist.Hist("Counts", dataset_axis, cats_axis, manual_axis),
            
            'cutflow': processor.defaultdict_accumulator(int),
            
        })

            
    @property
    def accumulator(self):
        return self._accumulator

    def process(self, df):
        
        output = self.accumulator.identity()
        #dataset = events.metadata['dataset']
        
        # ---- Define dataset ---- #
        dataset = df['dataset'] #coffea.processor.LazyDataFrame
        Dataset_info = df.available #list of available columns in LazyDataFrame object (Similar to 'Events->Show()' command in ROOT)
        
        # ---- Get triggers from Dataset_info ---- #
        #triggers = [itrig for itrig in Dataset_info if 'HLT_PFHT' in itrig]
        #AK8triggers = [itrig for itrig in Dataset_info if 'HLT_AK8PFHT' in itrig]

        # ---- Find numeric values in trigger strings ---- #
        #triggers_cut1 = [sub.split('PFHT')[1] for sub in triggers] # Remove string characters from left of number
        #triggers_cut2 = [sub.split('_')[0] for sub in triggers_cut1] # Remove string characters from right of number
        #isTriggerValue = [val.isnumeric() for val in triggers_cut2] # Boolean -> if string is only a number
        #triggers_cut2 = np.where(isTriggerValue, triggers_cut2, 0) # If string is not a number, replace with 0
        #triggers_vals = [int(val) for val in triggers_cut2] # Convert string numbers to integers
        
        #AK8triggers_cut1 = [sub.split('HT')[1] for sub in AK8triggers]
        #AK8triggers_cut2 = [sub.split('_')[0] for sub in AK8triggers_cut1]
        #isAK8TriggerValue = [val.isnumeric() for val in AK8triggers_cut2]
        #AK8triggers_cut2 = np.where(isAK8TriggerValue, AK8triggers_cut2, 0)
        #AK8triggers_vals = [int(val) for val in AK8triggers_cut2]
        
        # ---- Find Largest and Second Largest Value ---- #
        #triggers_vals.sort(reverse = True)
        #AK8triggers_vals.sort(reverse = True)
        
        #triggers_vals1 = str(triggers_vals[0])
        #triggers_vals2 = str(triggers_vals[1])
        #AK8triggers_vals1 = str(AK8triggers_vals[0])
        #AK8triggers_vals2 = str(AK8triggers_vals[1])
        
        # ---- Define strings for the selected triggers ---- #
        #HLT_trig1_str = [itrig for itrig in triggers if (triggers_vals1) in itrig][0]
        #HLT_trig2_str = [itrig for itrig in triggers if (triggers_vals2) in itrig][0]
        #HLT_AK8_trig1_str = [itrig for itrig in AK8triggers if (AK8triggers_vals1) in itrig][0]
        #HLT_AK8_trig2_str = [itrig for itrig in AK8triggers if (AK8triggers_vals2) in itrig][0]
        
        # ---- Define HLT triggers to be used ---- #
        #HLT_trig1 = df[HLT_trig1_str]
        #HLT_trig2 = df[HLT_trig2_str]
        #HLT_AK8_trig1 = df[HLT_AK8_trig1_str]
        #HLT_AK8_trig2 = df[HLT_AK8_trig2_str]
       
        
        # ---- Define AK8 Jets as FatJets ---- #
        FatJets = JaggedCandidateArray.candidatesfromcounts(
            df['nFatJet'],
            pt=df['FatJet_pt'],
            eta=df['FatJet_eta'],
            phi=df['FatJet_phi'],
            mass=df['FatJet_mass'],
            area=df['FatJet_area'],
            msoftdrop=df['FatJet_msoftdrop'],
            jetId=df['FatJet_jetId'],
            tau1=df['FatJet_tau1'],
            tau2=df['FatJet_tau2'],
            tau3=df['FatJet_tau3'],
            tau4=df['FatJet_tau4'],
            n3b1=df['FatJet_n3b1'],
            btagDeepB=df['FatJet_btagDeepB'],
            btagCSVV2=df['FatJet_btagCSVV2'],
            deepTag_TvsQCD=df['FatJet_deepTag_TvsQCD'],
            deepTagMD_TvsQCD=df['FatJet_deepTagMD_TvsQCD'],
            subJetIdx1=df['FatJet_subJetIdx1'],
            subJetIdx2=df['FatJet_subJetIdx2']
            )
        
        # ---- Define AK4 jets as Jets ---- #
        Jets = JaggedCandidateArray.candidatesfromcounts(
            df['nJet'],
            pt=df['Jet_pt'],
            eta=df['Jet_eta'],
            phi=df['Jet_phi'],
            mass=df['Jet_mass'],
            area=df['Jet_area']
            )
        # ---- Define SubJets ---- #
        SubJets = JaggedCandidateArray.candidatesfromcounts(
            df['nSubJet'],
            pt=df['SubJet_pt'],
            eta=df['SubJet_eta'],
            phi=df['SubJet_phi'],
            mass=df['SubJet_mass'],
            btagDeepB=df['SubJet_btagDeepB'],
            btagCSVV2=df['SubJet_btagCSVV2']
            )
        
        
        # ---- Get event weights from dataset ---- #
        if 'JetHT' in dataset: # If data is used...
            evtweights = np.ones(FatJets.size) # set all "data weights" to one
        else: # if Monte Carlo dataset is used...
            evtweights = df["Generator_weight"].reshape(-1, 1).flatten()
        
        # ---- Show all events ---- #
        output['cutflow']['all events'] += FatJets.size
        
        # ---- Apply Trigger(s) ---- #
        #FatJets = FatJets[HLT_AK8_trig1]
        #evtweights = evtweights[HLT_AK8_trig1]
        #Jets = Jets[HLT_AK8_trig1]
        #SubJets = SubJets[HLT_AK8_trig1]
        
        # ---- Jets that satisfy Jet ID ---- #
        jet_id = (FatJets.jetId > 0) # Loose jet ID
        FatJets = FatJets[jet_id]
        output['cutflow']['jet id'] += jet_id.any().sum()
        
        # ---- Apply pT Cut and Rapidity Window ---- #
        jetkincut_index = (FatJets.pt > self.ak8PtMin) & (np.abs(FatJets.p4.rapidity) < 2.4)
        FatJets = FatJets[ jetkincut_index ]
        output['cutflow']['jet kin'] += jetkincut_index.any().sum()
        
        # ---- Find two AK8 Jets ---- #
        twoFatJetsKin = (FatJets.counts == 2)
        FatJets = FatJets[twoFatJetsKin]
        evtweights = evtweights[twoFatJetsKin]
        Jets = Jets[twoFatJetsKin]
        SubJets = SubJets[twoFatJetsKin]
        output['cutflow']['two FatJets and jet kin'] += twoFatJetsKin.sum()
        
        # ---- Apply HT Cut ---- #
        hT = Jets.pt.sum()
        passhT = (hT > self.htCut)
        evtweights = evtweights[passhT]
        FatJets = FatJets[passhT]
        SubJets = SubJets[passhT]
        
        # ---- Randomly Assign AK8 Jets as TTbar Candidates 0 and 1 --- #
        if self.RandomDebugMode == True: # 'Sudo' randomizer for consistent results
            highPhi = FatJets.phi[:,0] > FatJets.phi[:,1]
            highRandIndex = np.where(highPhi, 0, 1)
            index = JaggedArray.fromcounts(np.ones(len(FatJets), dtype='i'), highRandIndex )
        else: # Truly randomize
            index = JaggedArray.fromcounts(np.ones(len(FatJets), dtype='i'), prng.randint(2, size=len(FatJets)))
        jet0 = FatJets[index] #J0
        jet1 = FatJets[1 - index] #J1
        
        ttbarcands = jet0.cross(jet1) #FatJets[:,0:2].distincts()
    
        # ---- Make sure we have at least 1 TTbar candidate pair and re-broadcast releveant arrays  ---- #
        oneTTbar = (ttbarcands.counts >= 1)
        output['cutflow']['>= one oneTTbar'] += oneTTbar.sum()
        ttbarcands = ttbarcands[oneTTbar]
        evtweights = evtweights[oneTTbar]
        FatJets = FatJets[oneTTbar]
        SubJets = SubJets[oneTTbar]
         
        # ---- Apply Delta Phi Cut for Back to Back Topology ---- #
        dPhiCut = (ttbarcands.i0.p4.delta_phi(ttbarcands.i1.p4) > 2.1).flatten()
        output['cutflow']['dPhi > 2.1'] += dPhiCut.sum()
        ttbarcands = ttbarcands[dPhiCut]
        evtweights = evtweights[dPhiCut]
        FatJets = FatJets[dPhiCut] 
        SubJets = SubJets[dPhiCut] 
        
        # ---- Identify subjets according to subjet ID ---- #
        hasSubjets0 = ((ttbarcands.i0.subJetIdx1 > -1) & (ttbarcands.i0.subJetIdx2 > -1))
        hasSubjets1 = ((ttbarcands.i1.subJetIdx1 > -1) & (ttbarcands.i1.subJetIdx2 > -1))
        GoodSubjets = ((hasSubjets0) & (hasSubjets1)).flatten()
   
        ttbarcands = ttbarcands[GoodSubjets]
        
        SubJets = SubJets[GoodSubjets]
        evtweights = evtweights[GoodSubjets]
       
        SubJet01 = SubJets[ttbarcands.i0.subJetIdx1] # FatJet i0 with subjet 1
        SubJet02 = SubJets[ttbarcands.i0.subJetIdx2] # FatJet i0 with subjet 2
        SubJet11 = SubJets[ttbarcands.i1.subJetIdx1] # FatJet i1 with subjet 1
        SubJet12 = SubJets[ttbarcands.i1.subJetIdx2] # FatJet i1 with subjet 2
        
        # ---- Define Rapidity Regions ---- #
        cen = np.abs(ttbarcands.i0.p4.rapidity - ttbarcands.i1.p4.rapidity) < 1.0
        fwd = (~cen)
        
        # ---- CMS Top Tagger Version 2 (SD and Tau32 Cuts) ---- #
        tau32_i0 = np.where(ttbarcands.i0.tau2>0,ttbarcands.i0.tau3/ttbarcands.i0.tau2, 0 )
        tau32_i1 = np.where(ttbarcands.i1.tau2>0,ttbarcands.i1.tau3/ttbarcands.i1.tau2, 0 )
        taucut_i0 = tau32_i0 < self.tau32Cut
        taucut_i1 = tau32_i1 < self.tau32Cut
        mcut_i0 = (self.minMSD < ttbarcands.i0.msoftdrop) & (ttbarcands.i0.msoftdrop < self.maxMSD) 
        mcut_i1 = (self.minMSD < ttbarcands.i1.msoftdrop) & (ttbarcands.i1.msoftdrop < self.maxMSD) 

        ttag_i0 = (taucut_i0) & (mcut_i0)
        ttag_i1 = (taucut_i1) & (mcut_i1)
        
        # ---- Define "Top Tag" Regions ---- #
        antitag = (~taucut_i0) & (mcut_i0) #Probe will always be ttbarcands.i1 (at)
        antitag_probe = np.logical_and(antitag, ttag_i1) #Found an antitag and ttagged probe pair for mistag rate (Pt)
        pretag =  ttag_i0 # Only jet0 (pret)
        ttag0 =   (~ttag_i0) & (~ttag_i1) # No tops tagged (0t)
        ttag1 =   ttag_i0 ^ ttag_i1 # Exclusively one top tagged (1t)
        ttagI =   ttag_i0 | ttag_i1 # At least one top tagged ('I' for 'inclusive' tagger; >=1t; 1t+2t)
        ttag2 =   ttag_i0 & ttag_i1 # Both jets top tagged (2t)
        Alltags = ttag0 | ttagI #Either no tag or at least one tag (0t+1t+2t)
        
        # ---- Pick FatJet that passes btag cut based on its subjet with the highest btag value ---- # 
        #btag_i0 = (ttbarcands.i0.btagCSVV2 > self.bdisc)
        #btag_i1 = (ttbarcands.i1.btagCSVV2 > self.bdisc)
        btag_i0 = ( np.maximum(SubJet01.btagCSVV2 , SubJet02.btagCSVV2) > self.bdisc )
        btag_i1 = ( np.maximum(SubJet11.btagCSVV2 , SubJet12.btagCSVV2) > self.bdisc )
        
        # --- Define "B Tag" Regions ---- #
        btag0 = (~btag_i0) & (~btag_i1) #(0b)
        btag1 = btag_i0 ^ btag_i1 #(1b)
        btag2 = btag_i0 & btag_i1 #(2b)
        
        # ---- Get Analysis Categories ---- # 
        # ---- They are (central, forward) cross (0b,1b,2b) cross (At,at,0t,1t,>=1t,2t) ---- #
        regs = [cen,fwd]
        btags = [btag0,btag1,btag2]
        ttags = [antitag_probe,antitag,pretag,ttag0,ttag1,ttagI,ttag2,Alltags]
        cats = [ (t&b&y).flatten() for t,b,y in itertools.product( ttags, btags, regs) ]
        labels_and_categories = dict(zip( self.anacats, cats ))
        
        # ---- Variables for Kinematic Histograms ---- #
        # ---- "i0" is the control jet, "i1" is the probe jet ---- #
        ttbarmass = ttbarcands.p4.sum().mass.flatten()
        jetpt = ttbarcands.i1.pt.flatten()
        jeteta = ttbarcands.i1.eta.flatten()
        jetphi = ttbarcands.i1.phi.flatten()
        jety = ttbarcands.i1.p4.rapidity.flatten()
        jetmass = ttbarcands.i1.p4.mass.flatten()
        SDmass = ttbarcands.i1.msoftdrop.flatten()
        jetdy = np.abs(ttbarcands.i0.p4.rapidity.flatten() - ttbarcands.i1.p4.rapidity.flatten())
        Tau32 = (ttbarcands.i1.tau3/ttbarcands.i1.tau2).flatten()
        # ---- Variables for Deep Tagger Analysis ---- #
        deepTag = ttbarcands.i1.deepTag_TvsQCD.flatten()
        deepTagMD = ttbarcands.i1.deepTagMD_TvsQCD.flatten()
        
        weights = evtweights.flatten()
        
        # ---- Define the SumW2 for MC Datasets ---- #
        output['cutflow']['sumw'] += np.sum(weights)
        output['cutflow']['sumw2'] += np.sum(weights**2)
        
        # ---- Define Momentum p of probe jet as the Mistag Rate variable; M(p) ---- #
        # ---- Transverse Momentum pT can also be used instead; M(pT) ---- #
        pT = ttbarcands.i1.pt.flatten()
        eta = ttbarcands.i1.eta.flatten()
        pz = np.sinh(eta)*pT
        p = np.absolute(np.sqrt(pT**2 + pz**2))
        
        # ---- Define the Numerator and Denominator for Mistag Rate ---- #
        numerator = np.where(antitag_probe, p, -1) # If no antitag and tagged probe, move event to useless bin
        denominator = np.where(antitag, p, -1) # If no antitag, move event to useless bin
        
        df = pd.DataFrame({"momentum":p}) # Used for finding values in LookUp Tables
        
        for ilabel,icat in labels_and_categories.items():
            ### ------------------------------------ Mistag Scaling ------------------------------------ ###
            if self.UseLookUpTables == True:
                # ---- Weight ttbar M.C. and data by mistag from data (corresponding to its year) ---- #
                if 'TTbar_' in dataset:
                    file_df = self.lu['JetHT' + dataset[-4:] + '_Data']['at' + str(ilabel[-5:])] #Pick out proper JetHT year mistag for TTbar sim.
                elif dataset == 'TTbar':
                    file_df = self.lu['JetHT']['at' + str(ilabel[-5:])] # All JetHT years mistag for TTbar sim.
                else:
                    file_df = self.lu[dataset]['at' + str(ilabel[-5:])] # get mistag (lookup) filename for 'at'
                    
                bin_widths = file_df['p'].values # collect bins as written in .csv file
                mtr = file_df['M(p)'].values # collect mistag rate as function of p as written in file
                wgts = mtr # Define weights based on mistag rates
                
                BinKeys = np.arange(bin_widths.size) # Use as label for BinNumber column in the new dataframe
                
                #Bins = pd.interval_range(start=0, periods=100, freq=100, closed='left') # Recreate the momentum bins from file_df as something readable for pd.cut()
                Bins = np.array(manual_bins)
                
                df['BinWidth'] = pd.cut(p, bins=Bins) # new dataframe column
                df['BinNumber'] = pd.cut(p, bins=Bins, labels=BinKeys)
                #df['BinNumber'] = pd.cut(p, bins=Bins).map(dict(zip(Bins,BinKeys))) # Use if Bins is defined by pd.interval_range
                
                BinNumber = df['BinNumber'].values # Collect the Bin Numbers into a numpy array
                BinNumber = BinNumber.astype('int64') # Insures the bin numbers are integers
            
                WeightMatching = wgts[BinNumber] # Match 'wgts' with corresponding p bin using the bin number
                Weights = weights*WeightMatching # Include 'wgts' with the previously defined 'weights'
            else:
                Weights = weights # No mistag rates, no change to weights
            ###---------------------------------------------------------------------------------------------###
            ### ----------------------------------- Mod-mass Procedure ------------------------------------ ###
            if self.ModMass == True:
                QCD_unweighted = util.load('TTbarResCoffea_QCD_unweighted_output.coffea')

                # ---- Extract event counts from QCD MC hist in signal region ---- #
                QCD_hist = QCD_unweighted['jetmass'].integrate('anacat', '2t' + str(ilabel[-5:])).integrate('dataset', 'QCD')
                data = QCD_hist.values() # Dictionary of values
                QCD_data = [i for i in data.values()][0] # place every element of the dictionary into a numpy array

                # ---- Re-create Bins from QCD_hist as Numpy Array ---- #
                bins = np.arange(510) #Re-make bins from the jetmass_axis starting with the appropriate range
                QCD_bins = bins[::10] #Finish re-making bins by insuring exactly 50 bins like the jetmass_axis

                # ---- Define Mod Mass Distribution ---- #
                ModMass_hist_dist = ss.rv_histogram([QCD_data,QCD_bins])
                jet1_modp4 = copy.copy(jet1.p4) #J1's Lorentz four vector that can be safely modified
                jet1_modp4["fMass"] = ModMass_hist_dist.rvs(size=jet1_modp4.size) #Replace J1's mass with random value of mass from mm hist
                ttbarcands_modmass = jet0.p4.cross(jet1_modp4) #J0's four vector x modified J1's four vector

                # ---- Apply Necessary Selections to new modmass version ---- #
                ttbarcands_modmass = ttbarcands_modmass[oneTTbar]
                ttbarcands_modmass = ttbarcands_modmass[dPhiCut]
                ttbarcands_modmass = ttbarcands_modmass[GoodSubjets]
                
                # ---- Manually sum the modmass p4 candidates (Coffea technicality) ---- #
                ttbarcands_modmass_p4_sum = (ttbarcands_modmass.i0 + ttbarcands_modmass.i1)
                
                # ---- Re-define Mass Variables for ModMass Procedure (pt, eta, phi are redundant to change) ---- #
                ttbarmass = ttbarcands_modmass_p4_sum.flatten().mass
                jetmass = ttbarcands_modmass.i1.mass.flatten()
            ###---------------------------------------------------------------------------------------------###
            output['cutflow'][ilabel] += np.sum(icat)
          
            output['ttbarmass'].fill(dataset=dataset, anacat=ilabel, 
                                ttbarmass=ttbarmass[icat],
                                weight=Weights[icat])
            output['jetpt'].fill(dataset=dataset, anacat=ilabel, 
                                jetpt=jetpt[icat],
                                weight=Weights[icat])
            output['probept'].fill(dataset=dataset, anacat=ilabel, 
                                jetpt=pT[icat],
                                weight=Weights[icat])
            output['probep'].fill(dataset=dataset, anacat=ilabel, 
                                jetp=p[icat],
                                weight=Weights[icat])
            output['jeteta'].fill(dataset=dataset, anacat=ilabel, 
                                jeteta=jeteta[icat],
                                weight=Weights[icat])
            output['jetphi'].fill(dataset=dataset, anacat=ilabel, 
                                jetphi=jetphi[icat],
                                weight=Weights[icat])
            output['jety'].fill(dataset=dataset, anacat=ilabel, 
                                jety=jety[icat],
                                weight=Weights[icat])
            output['jetdy'].fill(dataset=dataset, anacat=ilabel, 
                                jetdy=jetdy[icat],
                                weight=Weights[icat])
            output['numerator'].fill(dataset=dataset, anacat=ilabel, 
                                jetp=numerator[icat],
                                weight=Weights[icat])
            output['denominator'].fill(dataset=dataset, anacat=ilabel, 
                                jetp=denominator[icat],
                                weight=Weights[icat])
            output['jetmass'].fill(dataset=dataset, anacat=ilabel, 
                                   jetmass=jetmass[icat],
                                   weight=Weights[icat])
            output['SDmass'].fill(dataset=dataset, anacat=ilabel, 
                                   jetmass=SDmass[icat],
                                   weight=Weights[icat])
            output['tau32'].fill(dataset=dataset, anacat=ilabel,
                                          tau32=Tau32[icat],
                                          weight=Weights[icat])
            output['tau32_2D'].fill(dataset=dataset, anacat=ilabel,
                                          jetpt=pT[icat],
                                          tau32=Tau32[icat],
                                          weight=Weights[icat])
            output['deepTag_TvsQCD'].fill(dataset=dataset, anacat=ilabel,
                                          jetpt=pT[icat],
                                          tagger=deepTag[icat],
                                          weight=Weights[icat])
            output['deepTagMD_TvsQCD'].fill(dataset=dataset, anacat=ilabel,
                                            jetpt=pT[icat],
                                            tagger=deepTagMD[icat],
                                            weight=Weights[icat])
        
        return output

    def postprocess(self, accumulator):
        return accumulator

In [ ]:
filesets = {
    'ZprimeDM':zprimeDMfiles,
    'TTbar':ttbarfiles,
    'QCD':qcdfiles,
    'JetHT':jetdatafiles,
    'JetHT2016_Data':jetdatafiles2016,
    'JetHT2017_Data':jetdatafiles2017,
    'JetHT2018_Data':jetdatafiles2018
}

In [ ]:
tstart = time.time()

outputs_unweighted = {}

seed = 1234567890
prng = RandomState(seed)
Chunk = [100000, 600] # [chunksize, maxchunks]

for name,files in filesets.items(): 
    

    print(name)
    output = processor.run_uproot_job({name:files},
                                      treename='Events',
                                      processor_instance=TTbarResProcessor(UseLookUpTables=False,
                                                                           ModMass=False,
                                                                           RandomDebugMode=False,
                                                                           prng=prng),
                                      executor=processor.dask_executor,
                                      #executor=processor.iterative_executor,
                                      #executor=processor.futures_executor,
                                      executor_args={
                                          'client': client, 
                                          'nano':False, 
                                          'flatten':True, 
                                          'skipbadfiles':False,
                                          'workers': 2},
                                      chunksize=Chunk[0], maxchunks=Chunk[1]
                                     )

    elapsed = time.time() - tstart
    outputs_unweighted[name] = output
    print(output)
    util.save(output, 'TTbarResCoffea_' + name + '_unweighted_output_partial_2021_dask_run.coffea')

In [ ]:
print('Elapsed time = ', elapsed, ' sec.')
print('Elapsed time = ', elapsed/60., ' min.')
print('Elapsed time = ', elapsed/3600., ' hrs.') 

In [ ]:
for name,output in outputs_unweighted.items(): 
    print("-------Unweighted " + name + "--------")
    for i,j in output['cutflow'].items():        
        print( '%20s : %12d' % (i,j) )

In [ ]:
def mkdir_p(mypath):
    '''Creates a directory. equivalent to using mkdir -p on the command line'''

    from errno import EEXIST
    from os import makedirs,path

    try:
        makedirs(mypath)
    except OSError as exc: # Python >2.5
        if exc.errno == EEXIST and path.isdir(mypath):
            pass
        else: raise

In [ ]:
def DoesDirectoryExist(mypath): #extra precaution (Probably overkill...)
    '''Checks to see if Directory exists before running mkdir_p'''
    import os.path
    from os import path
    
    if path.exists(mypath):
        pass
    else:
        mkdir_p(mypath)

In [ ]:
import matplotlib.pyplot as plt
import warnings
import re # regular expressions
warnings.filterwarnings("ignore")

# ---- Reiterate categories ---- #
ttagcats = ["at"] #, "0t", "1t", "It", "2t"]
btagcats = ["0b", "1b", "2b"]
ycats = ['cen', 'fwd']

list_of_cats = [ t+b+y for t,b,y in itertools.product( ttagcats, btagcats, ycats) ]

In [ ]:
import os

In [ ]:
maindirectory = os.getcwd() # changes accordingly

In [ ]:
""" ---------------- CREATES MISTAG PLOTS ---------------- """
# ---- Only Use This Cell When LookUp Tables Are Not In Use (i.e. UseLookUpTables = False) ---- #

SaveDirectory = maindirectory + '/MistagPlots/'
DoesDirectoryExist(SaveDirectory) # no need to create the directory several times

# Function sqrt(x)
def forward(x):
    return x**(1/2)


def inverse(x):
    return x**2

print(SaveDirectory)
for iset in filesets:
    for icat in list_of_cats:
        print(iset)
        print(icat)
        title = iset + ' mistag ' + icat
        filename = 'mistag_' + iset + '_' + icat + '.' + 'png'
        print(outputs_unweighted[iset]['numerator'])
        Numerator = outputs_unweighted[iset]['numerator'].integrate('anacat', icat).integrate('dataset', iset)
        Denominator = outputs_unweighted[iset]['denominator'].integrate('anacat', icat).integrate('dataset', iset)
        print(Numerator)
        print(Denominator)
        mistag = hist.plotratio(num = Numerator, denom = Denominator,
                                error_opts={'marker': '.', 'markersize': 10., 'color': 'k', 'elinewidth': 1},
                                unc = 'num')
        plt.title(title)
        plt.ylim(bottom = 0, top = 0.12)
        plt.xlim(left = 100, right = 2500)
        
        # ---------- Better mistag plots are made in separate notebook ----------- #
        # ---- However, if one wants to save these plots right here, they may ---- #
        
        #plt.xticks(np.array([0, 500, 600, 700]))
        #mistag.set_xscale('function', functions=(forward, inverse))
        #mistag.set_xscale('log')
        #plt.savefig(SaveDirectory+filename, bbox_inches="tight")
        #print(filename + ' saved')

In [ ]:
""" ---------------- Scale-Factors for JetHT Data According to Year---------------- """
Nevts2016 = 625516390. # from dasgoclient
#Nevts2016_totalchunks = Nevts2016 / Chunk[0]
Nevts2016_sf = Nevts2016/outputs_unweighted['JetHT2016_Data']#Nevts2016_totalchunks / Chunk[1]

Nevts2017 = 410461585. # from dasgoclient
#Nevts2017_totalchunks = Nevts2017 / Chunk[0]
Nevts2017_sf = Nevts2017/outputs_unweighted['JetHT2017_Data']#Nevts2017_totalchunks / Chunk[1]

Nevts2018 = 676328827. # from dasgoclient
#Nevts2018_totalchunks = Nevts2018 / Chunk[0]
Nevts2018_sf = Nevts2018/outputs_unweighted['JetHT2018_Data']#Nevts2018_totalchunks / Chunk[1]

Nevts = Nevts2016 + Nevts2017 + Nevts2018
Nevts_totalchunks = Nevts / Chunk[0]
Nevts_sf = Nevts_totalchunks / Chunk[1]

print(Nevts2016_sf)
print(Nevts2017_sf)
print(Nevts2018_sf)
print(Nevts_sf)

In [ ]:
""" ---------------- Luminosities, Cross Sections, Scale-Factors ---------------- """ 
Lum2016 = 35920. # pb^-1 from https://twiki.cern.ch/twiki/bin/viewauth/CMS/PdmVAnalysisSummaryTable
Lum2017 = 41530.
Lum2018 = 59740.
Lum     = 137190.

ttbar_BR = 0.457 # 0.442 from PDG 2018
ttbar_xs = 1.0   # Monte Carlo already includes xs in event weight!! Otherwise, ttbar_xs = 831.76 * ttbar_BR  pb

ttbar2016_sf = ttbar_xs*Lum2016/(142155064.)
ttbar2017_sf = ttbar_xs*Lum2017/(142155064.)
ttbar2018_sf = ttbar_xs*Lum2018/(142155064.)
ttbar_sf = ttbar_xs*Lum/(142155064.)

print(ttbar2016_sf)
print(ttbar2017_sf)
print(ttbar2018_sf)
print(ttbar_sf)

qcd_xs = 1370000000.0 #pb From https://cms-gen-dev.cern.ch/xsdb
#qcd_sf = qcd_xs*Lum/18455107.

In [ ]:
""" ---------------- CREATE LOOK UP TABLE .CSV FILES ---------------- """
# ---- Only Use This Cell When LookUp Tables Are Not In Use for Previous Cell (UseLookUpTables = False) ---- #
from collections import defaultdict

runLUTS = True # Make separate Directory to place Look-Up Tables and perform ttbar subtraction for mistag weights

def multi_dict(K, type): # definition from https://www.geeksforgeeks.org/python-creating-multidimensional-dictionary/
    if K == 1: 
        return defaultdict(type) 
    else: 
        return defaultdict(lambda: multi_dict(K-1, type))
    
luts = {}
luts = multi_dict(2, str)

if runLUTS : 

    SaveDirectory = maindirectory + '/LookupTables/'
    DoesDirectoryExist(SaveDirectory)
    
    # ---- Check if TTbar simulation was used in previous processor ---- #
    if 'TTbar' in filesets:
        for iset in filesets:
            if iset != 'TTbar' or iset != 'QCD': # if JetHT filesets are found...
                print('\t\tfileset: ' + iset + '\n*****************************************************\n')
                for icat in list_of_cats:
                    title = iset + ' mistag ' + icat
                    filename = 'mistag_' + iset + '_' + icat + '.' + 'csv'

                    # ---- Info from TTbar ---- #
                    Numerator_tt = outputs_unweighted['TTbar']['numerator'].integrate('anacat',icat).integrate('dataset','TTbar')
                    Denominator_tt = outputs_unweighted['TTbar']['denominator'].integrate('anacat',icat).integrate('dataset','TTbar')
                    N_vals_tt = Numerator_tt.values()[()] #np.zeros( np.size(Numerator_tt.values()[()]) ) 
                    D_vals_tt = Denominator_tt.values()[()] #np.zeros( np.size(Denominator_tt.values()[()]) ) 

                    # ---- Info from JetHT datasets ---- #
                    Numerator = outputs_unweighted[iset]['numerator'].integrate('anacat',icat).integrate('dataset',iset)
                    Denominator = outputs_unweighted[iset]['denominator'].integrate('anacat',icat).integrate('dataset',iset)
                    N_vals = Numerator.values()[()]
                    D_vals = Denominator.values()[()]

                    # ---- Properly scale chunks of data and ttbar MC according to year of dataset used---- #
                    if '2016' in iset:
                        N_vals *= Nevts2016_sf 
                        D_vals *= Nevts2016_sf
                        N_vals_tt *= ttbar2016_sf
                        D_vals_tt *= ttbar2016_sf
                    elif '2017' in iset:
                        N_vals *= Nevts2017_sf 
                        D_vals *= Nevts2017_sf
                        N_vals_tt *= ttbar2017_sf
                        D_vals_tt *= ttbar2017_sf
                    elif '2018' in iset:
                        N_vals *= Nevts2018_sf 
                        D_vals *= Nevts2018_sf
                        N_vals_tt *= ttbar2018_sf
                        D_vals_tt *= ttbar2018_sf
                    else: # all years
                        N_vals *= Nevts_sf 
                        D_vals *= Nevts_sf
                        N_vals_tt *= ttbar_sf
                        D_vals_tt *= ttbar_sf

                    # ---- Subtract ttbar MC probe momenta from datasets' ---- #
                    N_vals_diff = np.abs(N_vals-N_vals_tt)
                    D_vals_diff = np.abs(D_vals-D_vals_tt)

                    print(N_vals_diff)
                    print(D_vals_diff)
                    print()

                    # ---- Define Mistag values ---- #
                    mistag_vals = np.where(D_vals_diff > 0, N_vals_diff/D_vals_diff, 0)
                    
                    # ---- Define Momentum values ---- #
                    p_vals = []
                    for iden in Numerator.identifiers('jetp'):
                        p_vals.append(iden)

                    # ---- Display and Save Dataframe, df, as Look-up Table ---- #
                    print('fileset:  ' + iset)
                    print('category: ' + icat)
                    print('________________________________________________\n')

                    d = {'p': p_vals, 'M(p)': mistag_vals} # 'data'

                    print("d vals = ", d)
                    print()
                    df = pd.DataFrame(data=d)
                    luts[iset][icat] = df

                    with pd.option_context('display.max_rows', None, 'display.max_columns', None): 
                        print(df)
                    print('\n')

                    df.to_csv(SaveDirectory+filename) # use later to collect bins and weights for re-scaling
            else: # If iset is not JetHT...
                for icat in list_of_cats:
                    Numerator = outputs_unweighted[iset]['numerator'].integrate('anacat',icat).integrate('dataset',iset)
                    Denominator = outputs_unweighted[iset]['denominator'].integrate('anacat',icat).integrate('dataset',iset)
                    N_vals = Numerator.values()[()]
                    D_vals = Denominator.values()[()]
                    print(N_vals)
                    print(D_vals)
                    print()
                    mistag_vals = np.where(D_vals > 0, N_vals/D_vals, 0)

                    p_vals = [] # Momentum values
                    for iden in Numerator.identifiers('jetp'):
                        p_vals.append(iden)
                    print('fileset:  ' + iset)
                    print('category: ' + icat)
                    print('________________________________________________\n')
                    d = {'p': p_vals, 'M(p)': mistag_vals}

                    print("d vals = ", d)
                    print()
                    df = pd.DataFrame(data=d)
                    luts[iset][icat] = df

                    with pd.option_context('display.max_rows', None, 'display.max_columns', None): 
                        print(df)
                    print('\n')

                    df.to_csv(SaveDirectory+filename) # use later to collect bins and weights for re-scaling

    else: # If iset did not run over 'TTbar' Simulation...
        for iset in filesets:
            print('\t\tfileset: ' + iset + '\n*****************************************************\n')
            for icat in list_of_cats:
                Numerator = outputs_unweighted[iset]['numerator'].integrate('anacat',icat).integrate('dataset',iset)
                Denominator = outputs_unweighted[iset]['denominator'].integrate('anacat',icat).integrate('dataset',iset)
                N_vals = Numerator.values()[()]
                D_vals = Denominator.values()[()]
                print(N_vals)
                print(D_vals)
                print()
                
                mistag_vals = np.where(D_vals > 0, N_vals/D_vals, 0)

                p_vals = []
                for iden in Numerator.identifiers('jetp'):
                    p_vals.append(iden)
                    
                print('fileset:  ' + iset)
                print('category: ' + icat)
                print('________________________________________________\n')
                d = {'p': p_vals, 'M(p)': mistag_vals}

                print("d vals = ", d)
                print()
                df = pd.DataFrame(data=d)
                luts[iset][icat] = df

                with pd.option_context('display.max_rows', None, 'display.max_columns', None): 
                    print(df)
                print('\n')

                df.to_csv(SaveDirectory+filename) # use later to collect bins and weights for re-scaling
            
else : # If runLUTS = False...
    for iset in filesets:
        print('\t\tfileset: ' + iset + '\n*****************************************************\n')
        for icat in list_of_cats:
            title = iset + ' mistag ' + icat
            filename = 'mistag_' + iset + '_' + icat + '.' + 'csv'
            luts[iset][icat] = pd.read_csv(filename)
print(luts)

In [ ]:
print("Events/s:", output['cutflow']['all events']/elapsed)

In [ ]:
filesets = {
    'ZprimeDM':zprimeDMfiles,
    'TTbar_2016':ttbarfiles, # TTbar MC to be weighted with mistag from JetHT2016_Data
    'TTbar_2017':ttbarfiles, # TTbar MC to be weighted with mistag from JetHT2017_Data
    'TTbar_2018':ttbarfiles, # TTbar MC to be weighted with mistag from JetHT2018_Data
    'TTbar':ttbarfiles,
    'QCD':qcdfiles,
    'JetHT':jetdatafiles,
    'JetHT2016_Data':jetdatafiles2016,
    'JetHT2017_Data':jetdatafiles2017,
    'JetHT2018_Data':jetdatafiles2018
}

In [ ]:
""" Runs Processor, Weights Datasets with Corresponding Mistag Weight """

tstart = time.time()

outputs_weighted = {}
prng = RandomState(seed)
Chunk = [100000, 600] # [chunksize, maxchunks]

for name,files in filesets.items(): 
    

    print(name)
    output = processor.run_uproot_job({name:files},
                                      treename='Events',
                                      processor_instance=TTbarResProcessor(UseLookUpTables=True,
                                                                           ModMass = False,
                                                                           lu=luts,
                                                                           prng=prng),
                                      executor=processor.dask_executor,
                                      #executor=processor.iterative_executor,
                                      #executor=processor.futures_executor,
                                      executor_args={
                                          'client': client, 
                                          'nano':False, 
                                          'flatten':True, 
                                          'skipbadfiles':False,
                                          'workers': 2}
                                      #chunksize=Chunk[0], maxchunks=Chunk[1]
                                     )

    elapsed = time.time() - tstart
    outputs_weighted[name] = output
    print(output)
    util.save(output, 'TTbarResCoffea_' + name + '_weighted_output_partial_2021_dask_run.coffea')


In [ ]:
for name,output in outputs_weighted.items(): 
    print("-------weighted " + name + "--------")
    for i,j in output['cutflow'].items():        
        print( '%20s : %12d' % (i,j) )

In [ ]:
""" Runs Processor, Weights Datasets with Corresponding Mistag Weight, Implements Mass Modification Procedure """

tstart = time.time()

outputs_weighted = {}
prng = RandomState(seed)
Chunk = [100000, 600] # [chunksize, maxchunks]

for name,files in filesets.items(): 
    

    print(name)
    output = processor.run_uproot_job({name:files},
                                      treename='Events',
                                      processor_instance=TTbarResProcessor(UseLookUpTables=True,
                                                                           ModMass = True,
                                                                           lu=luts,
                                                                           prng=prng),
                                      executor=processor.dask_executor,
                                      #executor=processor.iterative_executor,
                                      #executor=processor.futures_executor,
                                      executor_args={
                                         # 'client': client, 
                                          'nano':False, 
                                          'flatten':True, 
                                          'skipbadfiles':False,
                                          'workers': 2}
                                      #chunksize=Chunk[0], maxchunks=Chunk[1]
                                     )

    elapsed = time.time() - tstart
    outputs_weighted[name] = output
    print(output)
    util.save(output, 'TTbarResCoffea_' + name + '_ModMass_weighted_output_partial_2021_dask_run.coffea')